In [1]:
# Import Statements
import eeg_utils
import numpy as np
from eeg_utils import *
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
# Load EEG Data
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [3]:
# Visualize EEG Data Shapes
print("Training/Valid data shape: {}".format(X_train_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Training/Valid target shape: {}".format(y_train_valid.shape))
print("Test target shape: {}".format(y_test.shape))
print("Person train/valid shape: {}".format(person_train_valid.shape))
print("Person test shape: {}".format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [4]:
# Preprocess and Split Training Data
data_config = {"Duplicate": 2, "Mu": 0.0, "Sigma": 0.5, "Time_Point": 500}
X_train_valid, y_train_valid, X_test, y_test = vae_preprocess_data(X_train_valid,
                                                               y_train_valid,
                                                               X_test,
                                                               y_test, 
                                                               data_config)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid,
                                                      y_train_valid,
                                                      test_size=0.2,
                                                      random_state=0)

In [5]:
# Load Optimal Model
model_name = "./VAE_Train_All"
optim_model = keras.models.load_model(model_name) # Model that was trained on all subjects

In [6]:
# Evaluate Results (Train on all subjects, Test on all subjects)
optim_model.evaluate(X_train, y_train) # 52.60%
optim_model.evaluate(X_valid, y_valid) # 48.46%
optim_model.evaluate(X_test, y_test) # 49.44%

14/14 [==============================] - 3s 197ms/step - loss: 1.2333 - accuracy: 0.4944


[1.233320713043213, 0.4943566620349884]

In [7]:
# Separate data by patient
patient_data = []
patient_truths = []

for patient in np.unique(person_test):
    idx = np.where(person_test == patient)[0]
    patient_data.append(X_test[idx, :, :])
    patient_truths.append(y_test[idx])
    pass

In [8]:
# Evaluate model on each patient (Train on all subjects, Test on a single subject)
for i in range(len(patient_data)):
    optim_model.evaluate(patient_data[i], patient_truths[i])

2/2 [==============================] - 0s 100ms/step - loss: 1.2260 - accuracy: 0.4681


In [9]:
'''
Model Parameters: 11 Epochs
    - Data augmented VAE-CNN
    - Regularization : 0.006
    - Learning Rate : 0.001
    - Time Points : 500
    - Kernel_dim : (10, 1)
    - Channel_dim : 22
    - Dropout : 0.5
    - Duplicate : 2
    - Sigma : 0.5
    - Mu : 0.0
    
    N.B. For importance of the parameters above, see VAE_CNN Model notebook for architecture
'''

'\nModel Parameters: 11 Epochs\n    - Data augmented VAE-CNN\n    - Regularization : 0.006\n    - Learning Rate : 0.001\n    - Time Points : 500\n    - Kernel_dim : (10, 1)\n    - Channel_dim : 22\n    - Dropout : 0.5\n    - Duplicate : 2\n    - Sigma : 0.5\n    - Mu : 0.0\n    \n    N.B. For importance of the parameters above, see VAE_CNN Model notebook for architecture\n'